In [1]:
pip install gspread gspread-dataframe oauth2client


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install selenium webdriver-manager pandas gspread gspread-dataframe oauth2client


Note: you may need to restart the kernel to use updated packages.


In [5]:
# combined_scrape_to_gsheets.py
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials

# ----------------------------
# Configuration - edit if needed
# ----------------------------
SERVICE_ACCOUNT_JSON = "promising-cairn-468110-m0-d9e0ad1907f7.json"  # your downloaded JSON file
GOOGLE_SHEET_URL = "https://docs.google.com/spreadsheets/d/1dtdcGA53rDzNFDfRMsD9_PWcE-lDcwrzVD5Hx8GR8dM/edit"
HEADLESS = True
JS_WAIT_SECONDS = 5  # increase if your network is slow / site loads slowly

# Cricmetric URLs (change player name or format terms if you want another player)
PLAYER_NAME_ENCODED = "Virat%20Kohli"  # URL-encoded player name
urls = {
    "Test": f"https://www.cricmetric.com/sage/?q={PLAYER_NAME_ENCODED}%20performance%20year%20wise",
    "ODI":  f"https://www.cricmetric.com/sage/?q={PLAYER_NAME_ENCODED}%20ODI%20performance%20year%20wise",
    "T20I": f"https://www.cricmetric.com/sage/?q={PLAYER_NAME_ENCODED}%20T20I%20performance%20year%20wise",
    "IPL":  f"https://www.cricmetric.com/sage/?q={PLAYER_NAME_ENCODED}%20IPL%20performance%20year%20wise",
}

# ----------------------------
# Helper: start Selenium WebDriver
# ----------------------------
def start_driver(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless=new")  # newer headless; if issues try "--headless"
        options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# ----------------------------
# Scrape function
# ----------------------------
def scrape_cricmetric_tables(driver, url, wait_seconds=JS_WAIT_SECONDS):
    driver.get(url)
    time.sleep(wait_seconds)  # allow JS to load content
    # pandas can parse tables from the fully rendered page source
    try:
        tables = pd.read_html(driver.page_source)
    except ValueError:
        # pd.read_html raises ValueError if no tables found
        tables = []
    return tables

# ----------------------------
# Main flow: scrape all formats and combine
# ----------------------------
def main():
    driver = start_driver(HEADLESS)
    all_dfs = []

    try:
        for fmt, url in urls.items():
            print(f"[+] Fetching {fmt} from: {url}")
            tables = scrape_cricmetric_tables(driver, url)
            if not tables:
                print(f"    - No table found for {fmt}.")
                continue

            # assume first table is the relevant one (inspect if site structure changes)
            df = tables[0].copy()
            df["Format"] = fmt
            # optional: add source info or scrape timestamp
            df["Scraped_At"] = pd.Timestamp.now(tz="UTC")
            all_dfs.append(df)
            print(f"    - Got {len(df)} rows for {fmt}.")

    finally:
        driver.quit()

    if not all_dfs:
        print("No data was scraped for any format. Exiting.")
        return

    combined_df = pd.concat(all_dfs, ignore_index=True)
    # Optional: reorder columns so Format, Scraped_At appear first
    cols = [c for c in combined_df.columns if c not in ("Format", "Scraped_At")]
    combined_df = combined_df[["Format", "Scraped_At"] + cols]

    # Save a local copy (optional)
    combined_df.to_csv("Virat_Kohli_All_Formats_Performance.csv", index=False)
    print("[+] Combined CSV saved locally: Virat_Kohli_All_Formats_Performance.csv")

    # ----------------------------
    # Push to Google Sheets
    # ----------------------------
    # Scopes: Sheets + Drive (gspread uses Drive for file discovery)
    scope = ["https://spreadsheets.google.com/feeds",
             "https://www.googleapis.com/auth/drive"]

    creds = ServiceAccountCredentials.from_json_keyfile_name(SERVICE_ACCOUNT_JSON, scope)
    client = gspread.authorize(creds)

    # open by URL to avoid name mismatches
    sh = client.open_by_url(GOOGLE_SHEET_URL)
    worksheet = sh.sheet1  # first tab

    # clear existing worksheet contents (optional)
    worksheet.clear()

    # write dataframe to sheet
    set_with_dataframe(worksheet, combined_df)
    print(f"[+] Data pushed to Google Sheet: {GOOGLE_SHEET_URL}")

if __name__ == "__main__":
    main()


[+] Fetching Test from: https://www.cricmetric.com/sage/?q=Virat%20Kohli%20performance%20year%20wise


C:\Users\kk\AppData\Local\Temp\ipykernel_9436\1314400564.py:52: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(driver.page_source)


    - Got 16 rows for Test.
[+] Fetching ODI from: https://www.cricmetric.com/sage/?q=Virat%20Kohli%20ODI%20performance%20year%20wise


C:\Users\kk\AppData\Local\Temp\ipykernel_9436\1314400564.py:52: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(driver.page_source)


    - Got 19 rows for ODI.
[+] Fetching T20I from: https://www.cricmetric.com/sage/?q=Virat%20Kohli%20T20I%20performance%20year%20wise


C:\Users\kk\AppData\Local\Temp\ipykernel_9436\1314400564.py:52: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(driver.page_source)


    - Got 15 rows for T20I.
[+] Fetching IPL from: https://www.cricmetric.com/sage/?q=Virat%20Kohli%20IPL%20performance%20year%20wise


C:\Users\kk\AppData\Local\Temp\ipykernel_9436\1314400564.py:52: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(driver.page_source)


    - Got 19 rows for IPL.
[+] Combined CSV saved locally: Virat_Kohli_All_Formats_Performance.csv
[+] Data pushed to Google Sheet: https://docs.google.com/spreadsheets/d/1dtdcGA53rDzNFDfRMsD9_PWcE-lDcwrzVD5Hx8GR8dM/edit
